<a href="https://colab.research.google.com/github/estebanhernandezr/Entregas-Semanales/blob/master/SantiagoEsteban_AlvarezHernandez_ML_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Imports**

La dos celdas a continuación deben ejecutarse antes de realizar los **imports** para actualizar la versión de **pandas**.

In [ ]:
pip install --upgrade xlrd 

     |████████████████████████████████| 96 kB 5.4 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [ ]:
pip install scikit-learn-extra

     |████████████████████████████████| 1.7 MB 14.1 MB/s 


In [ ]:
#Para la visualizion y manipulacion de los datos 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

#Funcionamiento asociado al Aprendizaje de Máquina No Supervisado 
from sklearn_extra.cluster import KMedoids
from sklearn.datasets import load_wine
import scipy.cluster.hierarchy as sch
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_samples, silhouette_score

# **Importación y Preprocesamiento de los datos**

Especificamos las **rutas** de los archivos para importarlos a **python**.

In [ ]:
data_ratings_path = '/content/u_data_ratings.xls'
movies_genre_path = '/content/u_movies_genre.xls'
movies_items_path = '/content/u_movies_items.xls'
occupations_path  = '/content/u_occupation.xls'
user_info_path    = '/content/u_user_info.xls'

A continuación, importamos los archivos excel a python. Además de importar los archivos excel que contienen la inforamción, limpiamos y preprocesamos los datos para facilitar su entendimiento.

In [ ]:
#Importación del archivo u_data_ratings.xls 
data_ratings = pd.read_excel(data_ratings_path, names=['User ID', 'Movie ID', 'Ratings', 'Timestamp'])
data_ratings.drop(columns=['Timestamp'], inplace=True)

#Importación del archivo u_movies_genre.xls 
movies_genre = pd.read_excel(movies_genre_path,header=None, names=['Genre', 'Genre ID'])

#Importación y limpieza del archivo u_movies_items.xls 
movies_items = pd.read_excel(movies_items_path,header=None, names=["Movie ID", "Movie Title", "Release Date", "Video Release Date", "IMDb URL", "Unknown", "Action", "Adventure", "Animation", "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"])
movies_items.drop(columns=['Video Release Date','IMDb URL','Release Date'], inplace=True)
movies_items.dropna(inplace=True)

#Importación del archivo u_occupation.xls
occupations = pd.read_excel(occupations_path,header=None, names=["Occupation"])

#Importación del archivo u_user_info.xls
user_info  = pd.read_excel(user_info_path,header=None, names=["User ID", "Age", "Gender", "Occupation", "Zip Code"])

La limpieza de los archivos consistió en eliminar ciertas caracteristicas debido a que o no tienen inforamción o consideramos que no son relevantes para el desarrollo del proyecto. Habiendo hecho las respectivas importaciones y limpiezas de los datos, obtenemos las 5 tablas.

In [ ]:
data_ratings.head()

,User ID,Movie ID,Ratings
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [ ]:
movies_genre.head()

,Genre,Genre ID
0,unknown,0
1,Action,1
2,Adventure,2
3,Animation,3
4,Children's,4


In [ ]:
movies_items.head(-1)

,Movie ID,Movie Title,Unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1676,1677,Sweet Nothing (1995),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1677,1678,Mat' i syn (1997),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [ ]:
occupations.head()

,Occupation
0,administrator
1,artist
2,doctor
3,educator
4,engineer


In [ ]:
user_info.head(-1)

,User ID,Age,Gender,Occupation,Zip Code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
...,...,...,...,...,...
937,938,38,F,technician,55038
938,939,26,F,student,33319
939,940,32,M,administrator,2215
940,941,20,M,student,97229


A continaución analizaremos un poco la inforamción presente en las tablas para conseguir conocimeinto de dicha inforamción y poder hacer inferencia sobre los datos.

In [ ]:
data_ratings["Ratings"].describe()

count    65535.000000
mean         3.533883
std          1.130528
min          1.000000
25%          3.000000
50%          4.000000
75%          4.000000
max          5.000000
Name: Ratings, dtype: float64

In [ ]:
movies_items.iloc[:, 2:].apply(pd.Series.value_counts).drop([0], axis=0)

,Unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
1,2,251,135,42,122,505,109,50,725,22,24,92,56,61,247,101,251,71,27


In [ ]:
user_info["Age"].describe()

count    943.000000
mean      34.051962
std       12.192740
min        7.000000
25%       25.000000
50%       31.000000
75%       43.000000
max       73.000000
Name: Age, dtype: float64

In [ ]:
user_info["Occupation"].value_counts()

student          196
other            105
educator          95
administrator     79
engineer          67
programmer        66
librarian         51
writer            45
executive         32
scientist         31
artist            28
technician        27
marketing         26
entertainment     18
healthcare        16
retired           14
lawyer            12
salesman          12
none               9
homemaker          7
doctor             7
Name: Occupation, dtype: int64

In [ ]:
user_info["Gender"].value_counts()

M    670
F    273
Name: Gender, dtype: int64

Con esto en cuenta podemos concluir lo siguiente:

- Con respecto a los ratings, aunque sea de 0 a 5, el valor minimo es 1 mientras que el máximo es 5. Adicionalmente se puede vere que el puntaje promedio es 3.5.
- De los 18 géneros, los top 3 son
  1. Drama con 725
  2. Comedia con 505
  3. Accion y Thriller con 251
- De los 18 géneros, omitiendo el género desconocido, Fantasía es el género con menos peliculas con solo 22.
- De los usuarios, la edad promedio es 34, la edad mínima es 7 y la máxima es 73. También se tiene que la mayoria de los usuarios son estudiantes y 71% de los usuarios son hombres.

## **mini modelo relacional**

A continuación, ensamblamos un **modelo relacional de bases de datos** vinculado por medio de las columnas *User ID* y *Movie ID*. 

Muy brevemente, un **modelo relacional de base de datos** es un conjunto de tablas separadas (con diferentes atributos -columnas-) que se vinculan entre sí por medio de una columna (atributo) en común. Algunas de estas tablas pueden vincular entre sí a múltiples tablas diferentes: a este tipo particular de tablas se le conoce como **relación**.

Lo que hacemos a continuación es considerar que las tablas **user_info** y **movies_items** están vinculadas por medio de la relación **data_ratings**: **user_info** y **data_ratings** comparten la columna *User ID* en común; **data_ratings** y **movies_items** comparten la columna *Movie ID* en común.

El objetivo de esto es relacionar los atributos propios de la tabla **user_info** con aquellos propios de la tabla **movies_items**, por ejemplo, las tuplas: *(Occupation, Movie Genre)*, *(User Gendre, Movie Genre)*, *(Age, Movie Genre)*, entre otras, que no se relacionan directamente, pero que, por medio de *User ID* y *Movie ID* se relacionan en **data_ratings**.

Ahora bien, en principio, tenemos cinco tablas de datos separadas:
1.   **movies_genre**
2.   **movies_items**
3.   **data_ratings**
4.   **user_info**
5.   **occupations**

Sin embargo, algunas de ellas se encuentran implícitas dentro de otras: para ser más especifico, **occupations** $\subseteq$ **user_info**, mientras que **movies_genre** $\subseteq$ **movies_items**. No en vano, la tabla contenida se puede obtener (deducir) perfectamente de la tabla contenedora. Este embebimiento de tablas posibilita ignorar, por ahora, aquellas tablas embebidas.

Así que, nos quedamos solo con las siguientes tablas:
1.   **movies_items**
2.   **data_ratings**
3.   **user_info**

Sobre la cuales vamos a aplicar el modelo relacional de bases de datos, como lo mencionamos arriba.




A continuación, explicitamos el embebimiento de la tabla **movies_genre** dentro de la tabla **movies_items**: lo que se hace es crear una fila diferente para cada una de las clasificaciones que se le asignó a un película por género. En ese orden de ideas, una fila de película clasificada como de género de "Acción", "Aventura" y "Comedia" se convierte en tres filas diferentes que clasifican a la película como género de "Acción", otra como género de "Aventura" y otra como género de "Comedia". Esto se hace para generar una relación uno a uno (casi que funcional) entre una película y un género. Luego, cuando asignemos a cada *User ID* y *Rating* su respectivo *Movie ID*, implicitamente se le asigna un *Rating* y un *User ID* a cada clasificación (género).

In [ ]:
def incorporar_genero_peliculas(genre):
    genre_c = genre.copy()
    
    ACCI = genre_c.loc[genre_c['Action'] == 1].drop(['Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'], axis=1)
    ACCI['Genre'] = ACCI['Action'].apply(lambda x: 'Action' if x==1 else 'Undefined')
    ACCI = ACCI.drop(['Action'], axis=1)
    ADVE = genre_c.loc[genre_c['Adventure'] == 1].drop(['Action', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'], axis=1)
    ADVE['Genre'] = ADVE['Adventure'].apply(lambda x: 'Adventure' if x==1 else 'Undefined')
    ADVE = ADVE.drop(['Adventure'], axis=1)
    ANIM = genre_c.loc[genre_c['Animation'] == 1].drop(['Adventure', 'Action', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'], axis=1)
    ANIM['Genre'] = ANIM['Animation'].apply(lambda x: 'Animation' if x==1 else 'Undefined')
    ANIM = ANIM.drop(['Animation'], axis=1)
    CHIL = genre_c.loc[genre_c["Children's"] == 1].drop(['Adventure', 'Animation', "Action", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'], axis=1)
    CHIL['Genre'] = CHIL["Children's"].apply(lambda x: "Children's" if x==1 else 'Undefined')
    CHIL = CHIL.drop(["Children's"], axis=1)
    COME = genre_c.loc[genre_c['Comedy'] == 1].drop(['Adventure', 'Animation', "Children's", 'Action', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'], axis=1)
    COME['Genre'] = COME['Comedy'].apply(lambda x: 'Comedy' if x==1 else 'Undefined')
    COME = COME.drop(['Comedy'], axis=1)
    CRIM = genre_c.loc[genre_c['Crime'] == 1].drop(['Adventure', 'Animation', "Children's", 'Comedy', 'Action', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'], axis=1)
    CRIM['Genre'] = CRIM['Crime'].apply(lambda x: 'Crime' if x==1 else 'Undefined')
    CRIM = CRIM.drop('Crime', axis=1)
    DOCU = genre_c.loc[genre_c['Documentary'] == 1].drop(['Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Action', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'], axis=1)
    DOCU['Genre'] = DOCU['Documentary'].apply(lambda x: 'Documentary' if x==1 else 'Undefined')
    DOCU = DOCU.drop('Documentary', axis=1)
    DRAM = genre_c.loc[genre_c['Drama'] == 1].drop(['Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Action', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'], axis=1)
    DRAM['Genre'] = DRAM['Drama'].apply(lambda x: 'Drama' if x==1 else 'Undefined')
    DRAM = DRAM.drop('Drama', axis=1)
    FANT = genre_c.loc[genre_c['Fantasy'] == 1].drop(['Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Action', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'], axis=1)
    FANT['Genre'] = FANT['Fantasy'].apply(lambda x: 'Fantasy' if x==1 else 'Undefined')
    FANT = FANT.drop('Fantasy', axis=1)
    NOIR = genre_c.loc[genre_c['Film-Noir'] == 1].drop(['Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Action', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'], axis=1)
    NOIR['Genre'] = NOIR['Film-Noir'].apply(lambda x: 'Film-Noir' if x==1 else 'Undefined')
    NOIR = NOIR.drop('Film-Noir', axis=1)
    HORR = genre_c.loc[genre_c['Horror'] == 1].drop(['Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Action', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'], axis=1)
    HORR['Genre'] = HORR['Horror'].apply(lambda x: 'Horror' if x==1 else 'Undefined')
    HORR = HORR.drop('Horror', axis=1)
    MUSI = genre_c.loc[genre_c['Musical'] == 1].drop(['Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Action', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'], axis=1)
    MUSI['Genre'] = MUSI['Musical'].apply(lambda x: 'Musical' if x==1 else 'Undefined')
    MUSI = MUSI.drop('Musical', axis=1)
    MYST = genre_c.loc[genre_c['Mystery'] == 1].drop(['Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Action', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'], axis=1)
    MYST['Genre'] = MYST['Mystery'].apply(lambda x: 'Mystery' if x==1 else 'Undefined')
    MYST = MYST.drop('Mystery', axis=1)
    ROMA = genre_c.loc[genre_c['Romance'] == 1].drop(['Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Action', 'Sci-Fi', 'Thriller', 'War', 'Western'], axis=1)
    ROMA['Genre'] = ROMA['Romance'].apply(lambda x: 'Romance' if x==1 else 'Undefined')
    ROMA = ROMA.drop('Romance', axis=1)
    SCIF = genre_c.loc[genre_c['Sci-Fi'] == 1].drop(['Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Action', 'Thriller', 'War', 'Western'], axis=1)
    SCIF['Genre'] = SCIF['Sci-Fi'].apply(lambda x: 'Sci-Fi' if x==1 else 'Undefined')
    SCIF = SCIF.drop('Sci-Fi', axis=1)
    THRI = genre_c.loc[genre_c['Thriller'] == 1].drop(['Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Action', 'War', 'Western'], axis=1)
    THRI['Genre'] = THRI['Thriller'].apply(lambda x: 'Thriller' if x==1 else 'Undefined')
    THRI = THRI.drop('Thriller', axis=1)
    WARS = genre_c.loc[genre_c['War'] == 1].drop(['Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'Action', 'Western'], axis=1)
    WARS['Genre'] = WARS['War'].apply(lambda x: 'War' if x==1 else 'Undefined')
    WARS = WARS.drop('War', axis=1)
    WEST = genre_c.loc[genre_c['Western'] == 1].drop(['Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Action'], axis=1)
    WEST['Genre'] = WEST['Western'].apply(lambda x: 'Western' if x==1 else 'Undefined')
    WEST = WEST.drop(['Western'], axis=1)

    new_items = pd.concat([ACCI, ADVE, ANIM, CHIL, COME, CRIM, DOCU, DRAM, FANT, NOIR, HORR, MUSI, MYST, ROMA, SCIF, THRI, WARS, WEST])
    #print(new_items)
    return new_items

A continuación, definímos la función que llevara a cabo el **merge** de dos tablas diferentes, *tab1* y *tab2*, respecto a un atributo pivote en común: *pivot*. El resultado de esta operación es una tabla relación que vincula los atributos de la tabla *tab1* con los atributos de la tabla *tab2* por medio del atributo *pivot*.

In [ ]:
def fusionar_por_atributo(tab1, tab2, pivot):
    tab1_c = tab1.copy()
    tab2_c = tab2.copy()

    new_tab = pd.merge(tab1, tab2, on=pivot)
    #print(new_tab)
    return new_tab

Finalmente, llevamos a cabo el embebimiento de la tabla **movies_genre** dentro de **movies_items** y en seguida creamos unas tablas relación que van a enlazar las tablas relacionadas con la relación, sin relacionar, aún, las tablas: **data_ratings** $+$ **movies_items**, por medio del atributo *Movie ID*, y **user_info** $+$ **data_ratings** por medio del atributo *User ID*.

Finalmente, relacionamos las tablas **user_info** y **movies_items** por medio de la relación **data_ratings**: **user_info** $+$ (**data_ratings** + **movies_items**), o recíprocamente, **movies_items** $+$ (**data_ratings** $+$ **user_info**).

El resultado de estas relaciones lo observamos más abajo.

In [ ]:
new_table = incorporar_genero_peliculas(movies_items)
rating_movie = fusionar_por_atributo(data_ratings, new_table, 'Movie ID')
rating_user = fusionar_por_atributo(data_ratings, user_info, 'User ID')
relation_table1 = fusionar_por_atributo(rating_movie, user_info, 'User ID')
relation_table2 = fusionar_por_atributo(rating_user, new_table, 'Movie ID')

Operación: **movies_genre** $\subseteq$ **movies_items**

Resultado: **new_table** (embebimiento de **movies_genre** dentro de **movies_items**)

In [ ]:
print(new_table)
print(new_table.shape)
new_table.loc[new_table['Movie ID'] == 101]

      Movie ID                    Movie Title  Unknown    Genre
1            2               GoldenEye (1995)        0   Action
3            4              Get Shorty (1995)        0   Action
16          17     From Dusk Till Dawn (1996)        0   Action
20          21  Muppet Treasure Island (1996)        0   Action
21          22              Braveheart (1995)        0   Action
...        ...                            ...      ...      ...
1187      1188           Young Guns II (1990)        0  Western
1246      1247               Bad Girls (1994)        0  Western
1453      1454    Angel and the Badman (1947)        0  Western
1454      1455             Outlaw, The (1943)        0  Western
1586      1587  Terror in a Texas Town (1958)        0  Western

[2891 rows x 4 columns]
(2891, 4)


,Movie ID,Movie Title,Unknown,Genre
100,101,Heavy Metal (1981),0,Action
100,101,Heavy Metal (1981),0,Adventure
100,101,Heavy Metal (1981),0,Animation
100,101,Heavy Metal (1981),0,Horror
100,101,Heavy Metal (1981),0,Sci-Fi


Operación: **data_rating** $+$ **new_table**

Resultado: **rating_movie** (relación entre **data_ratings** y **movies_items**)

In [ ]:
rating_movie

,User ID,Movie ID,Ratings,Movie Title,Unknown,Genre
0,196,242,3,Kolya (1996),0,Comedy
1,63,242,3,Kolya (1996),0,Comedy
2,226,242,5,Kolya (1996),0,Comedy
3,154,242,3,Kolya (1996),0,Comedy
4,306,242,5,Kolya (1996),0,Comedy
...,...,...,...,...,...,...
138720,655,1256,3,"Designated Mourner, The (1997)",0,Drama
138721,463,1606,2,Deceiver (1997),0,Crime
138722,798,1544,3,It Takes Two (1995),0,Comedy
138723,565,1622,4,"Paris, France (1993)",0,Comedy


Operación: **data_rating** + **user_info**

Resultado: **rating_user** (relación entre **data_ratings** y **user_info**)

In [ ]:
rating_user

,User ID,Movie ID,Ratings,Age,Gender,Occupation,Zip Code
0,196,242,3,49,M,writer,55105
1,196,393,4,49,M,writer,55105
2,196,381,4,49,M,writer,55105
3,196,251,3,49,M,writer,55105
4,196,655,5,49,M,writer,55105
...,...,...,...,...,...,...,...
65530,919,787,3,25,M,other,14216
65531,921,24,3,20,F,student,98801
65532,910,182,4,28,M,healthcare,29301
65533,910,1012,4,28,M,healthcare,29301


Operación: **user_info** $+$ (**data_rating** $+$ **movies_items**)

Resultado: **relation_table1** (relación entre **user_info** y **movies_items**, por medio del atributo *User ID*)

In [ ]:
relation_table1

,User ID,Movie ID,Ratings,Movie Title,Unknown,Genre,Age,Gender,Occupation,Zip Code
0,196,242,3,Kolya (1996),0,Comedy,49,M,writer,55105
1,196,257,2,Men in Black (1997),0,Action,49,M,writer,55105
2,196,257,2,Men in Black (1997),0,Adventure,49,M,writer,55105
3,196,257,2,Men in Black (1997),0,Comedy,49,M,writer,55105
4,196,257,2,Men in Black (1997),0,Sci-Fi,49,M,writer,55105
...,...,...,...,...,...,...,...,...,...,...
138720,913,99,4,Snow White and the Seven Dwarfs (1937),0,Musical,27,M,student,76201
138721,918,488,3,Sunset Blvd. (1950),0,Film-Noir,40,M,scientist,70116
138722,921,24,3,Rumble in the Bronx (1995),0,Action,20,F,student,98801
138723,921,24,3,Rumble in the Bronx (1995),0,Adventure,20,F,student,98801


Operación: **movies_items** $+$ (**data_rating** $+$ **user_info**)

Resultado: **relation_table2** (relación entre **user_info** y **movies_items**, por medio del atributo *Movie ID*)

In [ ]:
relation_table2

,User ID,Movie ID,Ratings,Age,Gender,Occupation,Zip Code,Movie Title,Unknown,Genre
0,196,242,3,49,M,writer,55105,Kolya (1996),0,Comedy
1,6,242,4,42,M,executive,98101,Kolya (1996),0,Comedy
2,234,242,4,60,M,retired,94702,Kolya (1996),0,Comedy
3,63,242,3,31,M,marketing,75240,Kolya (1996),0,Comedy
4,181,242,1,26,M,executive,21218,Kolya (1996),0,Comedy
...,...,...,...,...,...,...,...,...,...,...
138720,782,1652,1,21,F,artist,33205,Temptress Moon (Feng Yue) (1996),0,Romance
138721,798,1544,3,40,F,writer,64131,It Takes Two (1995),0,Comedy
138722,828,1646,4,28,M,librarian,85282,Men With Guns (1997),0,Action
138723,828,1646,4,28,M,librarian,85282,Men With Guns (1997),0,Drama


In [ ]:
relation_table1.equals(relation_table2)

False

# **Visualización de los datos**